In [2]:
# Turn off Tensorflow warnings

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Key imports

import numpy as np
import tensorflow as tf
import keras as k
import pickle
from math import prod
from sklearn.preprocessing import StandardScaler
from itertools import product as iprod
import random


## Helper functions

def load_object(filename):
    with open(filename, 'rb') as imp:  # Overwrites any existing file.
        res = pickle.load(imp)
    return res

def create_lstm_sample(scaled_sample):
    if scaled_sample.shape[0] % 5 == 0:
        s = scaled_sample
    else:
        s = scaled_sample[:-(scaled_sample.shape[0]%5)]
    return np.stack(np.array_split(s, scaled_sample.shape[0]//5), axis=0)

In [3]:
### Load objects 

lstm_test_data = load_object('lstm_test_data.pkl')
lstm_test_labels = load_object('lstm_test_labels.pkl')
lstm_test_data = np.concatenate([lstm_test_data[d] for d in ['browse','chat','mail','p2p']])
lstm_test_labels = np.concatenate([lstm_test_labels[d] for d in ['browse','chat','mail','p2p']])
scaler = load_object('scaler.pkl')

stacked_cnn_lstm_sae = tf.keras.saving.load_model('stacked_cnn_lstm_sae.53.keras')



In [5]:
### Get data for individual files/samples

import glob

data_path = "Data/"
browse_data_path = data_path + "browse/"
chat_data_path = data_path + "chat/"
mail_data_path = data_path + "mail/"
p2p_data_path =  data_path + "p2p/"
data_paths = [browse_data_path, chat_data_path, mail_data_path, p2p_data_path]

datasets = dict()
sample_data = dict()
sample_labels = dict()
scaled_sliced_data = dict()
test_samples = dict()

for (index, path) in  enumerate(data_paths):
    datasets[path] = list(glob.glob(path + "*.pcap"))
    sample_data[path] = []

    for sample_path in datasets[path]:
        s = np.loadtxt(sample_path, delimiter=',').reshape((-1,6))
        if s.shape[0] >= 25:
            sample_data[path].append(s)

# Scale and process for LSTM input
    
for path in data_paths:
    scaled_sliced_data[path] = []
    for s in sample_data[path]:
        scaled_sliced_data[path].append(create_lstm_sample(scaler.transform(s)))

# Choose a test sample, one which does not already fool the model
    
for (idx, path) in enumerate(data_paths):
    if idx == 2:
        continue
    correct = False
    while not correct:
        candidate_choice = scaled_sliced_data[path].pop()
        vals, counts = np.unique(stacked_cnn_lstm_sae.predict(candidate_choice).argmax(axis=-1), return_counts=True)
        mr = vals[np.argmax(counts)]
        print(mr, idx)
        correct = (mr == idx)
    test_samples[path] = candidate_choice


1/1 [==============================] - 0s 13ms/step
0 0
1/1 [==============================] - 0s 37ms/step
0 1
1/1 [==============================] - 0s 55ms/step
2 1
1/1 [==============================] - 0s 42ms/step
3 1
2/2 [==============================] - 0s 8ms/step
3 1
3/3 [==============================] - 0s 10ms/step
0 1
1/1 [==============================] - 0s 49ms/step
0 1
1/1 [==============================] - 0s 47ms/step
3 1
1/1 [==============================] - 0s 43ms/step
3 1
1/1 [==============================] - 0s 65ms/step
0 1
1/1 [==============================] - 0s 56ms/step
0 1
4/4 [==============================] - 0s 13ms/step
0 1
1/1 [==============================] - 0s 74ms/step
3 1
1/1 [==============================] - 0s 56ms/step
1 1
4/4 [==============================] - 0s 7ms/step
0 3
1/1 [==============================] - 0s 48ms/step
3 3


In [7]:
# p2p_sample_data = np.loadtxt("Data/p2p-sample.csv", delimiter=',')
# browse_sample_data = np.loadtxt("Data/browse-sample.csv", delimiter=',')
# chat_sample_data = np.loadtxt("Data/mail-sample.csv", delimiter=',')

# scaled_p2p_sample_data = scaler.transform(p2p_sample_data)
# scaled_browse_sample_data = scaler.transform(browse_sample_data)
# scaled_chat_sample_data = scaler.transform(chat_sample_data)

p2p_sample = test_samples['Data/p2p/']
browse_sample = test_samples['Data/browse/']
chat_sample = test_samples['Data/chat/']


p2p_label = np.tile(np.array([0.0, 0.0, 0.0, 1.0]),(1, 1))
browse_label = np.tile(np.array([1.0, 0.0, 0.0, 0.0]), (1, 1))
chat_label = np.tile(np.array([0.0, 1.0, 0.0, 0.0]), (1, 1))

In [8]:
## Additional helper functions

def get_jacobian(forward_derivative, idx, target_label):
    return forward_derivative[(target_label,) + idx]

def input_shape_range(shape):
    return iprod(*(range(s) for s in shape))

def make_result_sane(inp):
    result = np.copy(inp)
    result[0] = np.ceil(inp[0])
    result[1] = np.ceil(inp[1])
    result[2] = 100*result[0]
    result[3] = 8*result[1]*100
    result[4] = inp[4] if result[0] > 1 else 0
    result[5] = result[1]/result[0] if result[0] != 0 else 0
    return result
    
def predict(model, inp):
    # This is used to take the output of a softmax classifier of 
    # the type found in this paper
    # and get the prediction in terms of one-hot encoding.
    
    output_size = model.output_shape[1]
    res1 = model.predict(inp).argmax(axis=1)
    vals, counts = np.unique(res1, return_counts=True)
    res = vals[np.argmax(counts)]
    return res1, np.eye(output_size)[res]
    # output_size = model.output_shape[1]
    # res1 = tf.argmax(model(inp), axis=1)
    # vals, _, counts = tf.unique_with_counts(res1)
    # res = vals[tf.argmax(counts)]
    # return res1, np.eye(output_size)[res]

@tf.function
def predict_one(model, inp):
    res = tf.argmax(model(inp))[0]
    return res
    

@tf.function
def saliency_map(forward_derivative, input_shape, l, output_size):
    s = dict()
    for idx in input_shape_range(input_shape):
        g = get_jacobian(forward_derivative, idx, l)
        if g < 0:
            s[idx] = tf.experimental.numpy.float64(0)
        else:
            t = tf.experimental.numpy.float64(0)
            for j in range(output_size):
                if j != l:
                    t += get_jacobian(forward_derivative, idx, j)
            if t > 0:
                s[idx] = tf.experimental.numpy.float64(0)
            else:
                s[idx] = g*tf.abs(t)

    return s


def reverse_scaler(scaler, scaled_input):
    res = np.copy(scaled_input)
    if scaler.with_std:
        res *= scaler.scale_
    if scaler.with_mean:
        res += scaler.mean_
    return res

def scale(scaler):
    def _(inp):
        res = np.copy(inp)
        if scaler.with_mean:
            res -= scaler.mean_
        if scaler.with_std:
            res /= scaler.scale_
        return res
    return _


@tf.function
def jacobian(model, adversarial_sample, input_shape, output_size):
    with tf.GradientTape() as tape:
        a = adversarial_sample
        tape.watch(a)
        prediction = model(a)
    return tf.reshape(tape.jacobian(prediction, a),
                    (output_size,) + input_shape[1:])

## The key routine used to create adversarial examples here

def create_adversarial_sample(sample, model, target_label, max_distortion, theta):
    adversarial_sample = np.copy(sample)
    input_shape = sample.shape
    output_size = model.output_shape[1]
    delta = 0
    while (not np.equal(predict_one(model, adversarial_sample), target_label).all() and delta < max_distortion):
        forward_derivative = jacobian(model, adversarial_sample, input_shape, output_size)
        s = saliency_map(forward_derivative, input_shape[1:], target_label.argmax(), output_size)
        # Argmax over dict entries
        argmax_res, curr_max = None, None
        for idx in s.keys():
            if not curr_max or s[idx] > curr_max:
                curr_max = s[idx]
                argmax_res = idx
        adversarial_sample[0][argmax_res] += theta
        delta = np.linalg.norm(adversarial_sample - sample)
    return adversarial_sample
    
def create_adversarial_sample_lstm(sample, model, target_label, max_distortion, theta):
    results = []
    for timeseries_idx in range(sample.shape[0]):
        subsample = sample[timeseries_idx].reshape((1,5,6))
        adversarial_subsample = create_adversarial_sample(subsample, model, target_label, max_distortion, theta)
        results.append(adversarial_subsample)
    return np.array(results)

In [10]:
theta = 0.01
sample_label_combos = [(p2p_sample, browse_label), (browse_sample, p2p_label), (chat_sample, browse_label),
                       (p2p_sample, chat_label), (browse_sample, chat_label), (chat_sample, p2p_label)]

with open('jsma.log', 'a') as logfile:
    for (sample, label) in sample_label_combos:
        print("Target label:", label, file=logfile)
        print("Scaled Sample:", sample, file=logfile)
        r1, r = predict(stacked_cnn_lstm_sae, sample)
        print("Majority prediction:", r1, file=logfile)
        print("Individual predictions:", r, file=logfile)
        print("ADVERSARIAL SAMPLES:\n\n", file=logfile)
        for max_distortion in [0.1, 1, 10, 20, 50]:
            print("max distortion =", max_distortion, file=logfile)
            adv = create_adversarial_sample_lstm(sample, stacked_cnn_lstm_sae, label, max_distortion, theta)
            print("Adversarial raw:", adv, file=logfile)
            adv2 = adv.reshape((-1, 5, 6))
            adv3 = reverse_scaler(scaler, adv2)
            print("Unscaled adversarial:", adv3, file=logfile)
            adv4 = np.apply_along_axis(make_result_sane, 2, adv3)
            print("Unscaled adversarial made sane:", adv4, file=logfile)
            scaled_adversarial = np.apply_along_axis(scale(scaler), 2, adv4)
            print("Scaled adversarial made sane:", scaled_adversarial, file=logfile)
            print("Sample difference:", scaled_adversarial-sample, file=logfile)
            r1, r = predict(stacked_cnn_lstm_sae, scaled_adversarial)
            print("Majority prediction:", r1, file=logfile)
            print("Individual predictions:", r, file=logfile)
            print("\n", file=logfile)
        print("\n", file=logfile)
    print("END LOG", file=logfile)

1/1 [==============================] - 0s 24ms/step


In [ ]:
ad